In [1]:
import os

In [2]:
%pwd

'c:\\Users\\kshar\\myfiles\\projects\\Predicting_rain_using_ML\\research'

In [3]:
#change the directory to root directory
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\kshar\\myfiles\\projects\\Predicting_rain_using_ML'

In [5]:

# configuration of entity 

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from Predicting_rainfall_project.constants import *
from Predicting_rainfall_project.utils.common import read_yaml , create_directories

In [7]:
#configuration manager class

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config1 = self.config.data_ingestion

        create_directories([config1.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config1.root_dir,
            source_URL= config1.source_URL,
            local_data_file= config1.local_data_file,
            unzip_dir= config1.unzip_dir 
        )

        return data_ingestion_config



In [8]:
import os
import urllib.request as request
import zipfile
from Predicting_rainfall_project import logger
from Predicting_rainfall_project.utils.common import get_size

In [9]:
# DataIngestion_components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [10]:
# pipeline
try:
    # get the configuration details using config manager
    config = ConfigurationManager()
    # initialize the data ingestion component with the configurations
    data_ingestion_config = config.get_data_ingestion_config()
    # call the method of data ingestion component to read and process the data
    data_ingestion = DataIngestion(config=data_ingestion_config)
    # download the data
    data_ingestion.download_file()
    # unzip the file
    data_ingestion.extract_zip_file()
except Exception as e:
        raise e


[2024-01-30 18:15:48,114: DEBUG: common: yaml_file: config\config.yaml is successfully loaded:]
[2024-01-30 18:15:48,114: DEBUG: common: yaml_file: param.yaml is successfully loaded:]
[2024-01-30 18:15:48,121: DEBUG: common: yaml_file: schema.yaml is successfully loaded:]
[2024-01-30 18:15:48,121: DEBUG: common: artifacts is made at artifacts:]
[2024-01-30 18:15:48,121: DEBUG: common: artifacts\data_ingestion is made at artifacts/data_ingestion:]
[2024-01-30 18:15:49,435: INFO: 755459253: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 94185
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "6d04cd61337a0141686f48e75308d6759a6eaef1d1c2a8cc6ae045c02d64373c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8E22:42C2:33ED59:3D908D:65B8E6AD
Accept